In [ ]:
#!pip install git+https://github.com/rabernat/xmitgcm.git@fix-llcreader-klevels-bug

In [ ]:
import numpy as np
import xmitgcm.llcreader as llcreader
%matplotlib inline
import holoviews as hv
#from holoviews.operation.datashader import regrid
#hv.extension('bokeh')
import xarray as xr
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

%matplotlib inline
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)


In [ ]:
model = llcreader.ECCOPortalLLC2160Model()
model

In [ ]:
ds_sst = model.get_dataset(varnames=['Theta'], k_levels=[0], type='latlon')
ds_sst

In [ ]:
#create a daily climatology and anomaly
climatology_mean = ds_sst.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
ds_anomaly = ds_sst.groupby('time.dayofyear')-climatology_mean  #take out annual mean to remove trends
anom = ds_anomaly.Theta.isel(k=0,time=6200,j=slice(4900,5400),i=slice(4600,5200)).load()

In [ ]:
# okay, now in x direction smooth by radius, convolution ?
noise = 1
sst = ds_sst.Theta.isel(k=0,time=6200,j=slice(4900,5400),i=slice(4600,5200)).load()
#create random array 0 to 1
tem1a = (sst/sst)*np.random.rand(500,600)
#tem1a.plot()
#create linearly increasing random array
tem2 = (sst/sst).load()
for j in range(500):
    tem2[j,:]=tem2[j,:]*((j/500)**3*noise)
noise = tem1a*tem2
sstnoise = sst+sst*noise
anomsmooth=sstnoise.copy(deep=True).load()
noisesmooth=sstnoise.copy(deep=True).load()
for i in range(600):
    w = int(i/20)+1 #window for smoothing
    i1,i2=i-w+1,i+w
    if i1<0:
        i1=0
    if i2>599:
        i2=599
    for j in range(500):
        j1,j2=j-w+1,j+w
        if j1<0:
            j1=0
        if j2>499:
            j2=499
        anomsmooth[j,i]=sst[j1:j2,i1:i2].mean()
        noisesmooth[j,i]=noise[j1:j2,i1:i2].mean()
    if int(i/100)*100==i:
        print(i,600)
anomsmooth.to_netcdf('anomsmooth45a.nc')
noisesmooth.to_netcdf('anomsmooth45a.nc')

In [ ]:
plt.rcParams['figure.figsize'] = (18,10)
#(anomsmooth+noisesmooth*anomsmooth).plot(vmin=10,vmax=20,cmap='turbo')
#(anomsmooth).plot(vmin=-2,vmax=2,cmap='seismic')

xx = np.arange(600)/600*260 
xlabel = [20,60,100,140,180,220]
yy = np.arange(500)/500*30+5
ylabel = [10,20,30]
dotx = [35,250,50,62,80,200,20]
doty = [17,18,32,22,25,18,11]
plt.pcolormesh(xx,yy,anomsmooth+noisesmooth*anomsmooth,vmin=8,vmax=22,cmap='turbo')
plt.scatter(dotx,doty)
plt.tick_params(labelsize=30)
plt.xlabel('Spatial Scale (km)',size=30)
plt.ylabel('Random Uncertainty \n Latent + Sensible Heat Flux (%)',size=30)
plt.savefig('discovered_country_sst_smoothed_A_20210208.png')
plt.savefig('discovered_country_sst_smoothed_A_20210208.pdf')